### Imports

In [1]:
import mysql.connector, json, os, requests, re
from bs4 import BeautifulSoup
conn = mysql.connector.connect(user=os.environ['MYSQL_DB_USER'], password=os.environ['MYSQL_DB_PASSWORD'], host='127.0.0.1', database='etymology_explorer')
cursor = conn.cursor()

### Get urls

Ok so I want to have the data staged somewhere so that I can check it before I update everything else. I could move it into a dataframe. I already have one that stores a ton of information. The definitions could be one big array, or it could be it's own csv with definitions as a separate list. I think I'm going to do that first. It is more flexible. And I can do that for the pronunciations. It will be the same as the other DF. I think I could also just have it append all the results to a file
- word, language, etymology
- word, language, pronunciation
- word, language, pos, definition

Cat is a good test for multiple etymologies

In [151]:
import pandas as pd

# Get the list of word-language pairs
df = pd.read_csv('~/etymology_files/ety_master.csv', 
                 usecols = ['word', 'language'], 
                 converters={'word' : str, 'language': str}
                )

# Set all non-reconstructions to be 'None'
normal_language_rows = [not language.startswith('Proto') for language in df['language'].tolist()] #bools
df.loc[normal_language_rows, 'language'] = None
df = df.drop_duplicates()
url_terms = [row[0] if row[1] is None else 'Reconstruction:'+row[1]+'/'+row[0] for row in df.values]
urls = ['https://en.wiktionary.org/api/rest_v1/page/html/' + term for term in url_terms]

### Scrape Wiktionary Page

In [110]:
all_pos = ['adfix', 'adjective', 'adnoun', 'adverb', 'article', 'auxiliary verb', 'cardinal number', 'collective numeral',
           'conjunction', 'coverb', 'demonstrative determiner', 'demonstrative pronoun', 'determinative', 'determiner',
           'gerund', 'indefinite pronoun', 'infinitive', 'interjection', 'interrogative pronoun', 'intransitive verb',
           'noun', 'number', 'numeral', 'ordinal', 'ordinal number', 'part of speech', 'participle', 'particle',
           'personal pronoun', 'phrasal preposition', 'possessive adjective', 'possessive determiner', 'possessive pronoun',
           'postposition', 'preposition', 'preverb', 'pronoun', 'quasi-adjective', 'reciprocal pronoun', 'reflexive pronoun',
           'relative pronoun', 'speech disfluency', 'substantive', 'transitive', 'transitive verb', 'verb', 'verbal noun']

In [112]:
def getDefsFromPOS(ety_pronunc_pos_node):
    keep_def_tags = ('i', 'b', 'a', 'span', None)
    node_data = []
    for li in list(ety_pronunc_pos_node.parent.find('ol').children): # get defs from ordered list
        if li.name != 'li': continue # This is a newline tag

        if li.find('ol'): #Ordered list means the sub items are the definition
            for sub_li in list(li.find('ol').children):
                if sub_li.name != 'li': continue # Skip newline tags

                for child in sub_li.children:
                    if child.name not in keep_def_tags: child.clear() #Get rid of quotes and subitems

                node_data.append(sub_li.text.strip())

        else: # otherwise grab the text

            for child in li.children:
                if child.name not in keep_def_tags: child.clear() #Get rid of quotes and subitems
            node_data.append(li.text.strip())
    return node_data
    

In [113]:
word='cat'
response = requests.get(f'https://en.wiktionary.org/api/rest_v1/page/html/{word}'); response
page = response.url.replace('https://en.wiktionary.org/api/rest_v1/page/html/', '')
page = re.sub('Reconstruction:[^\/]+?\/(.*)', r'\1', page) #Remove reconstruction text if necessary

soup = BeautifulSoup(response.content, 'html.parser')

page_data = {'term': page} # Variable to store the data

for lang_node in soup.find_all('h2'): # Go through each Language
    language = lang_node.text.strip()
    language_entries = [{}]

    for ety_pronunc_pos_node in lang_node.parent.find_all('h3'): # Go through each ety,pronu, or pos
        node_data = []
        node_class = re.sub('_\d+| \d+', '', ety_pronunc_pos_node.text).lower() #removed '_x' info
#         node_text = ety_pronunc_pos_node.text.lower()

        if node_class == 'etymology':
            #Only looking at the first <p> element for etymology text
            entry_data = {'etymology': ety_pronunc_pos_node.parent.find('p').text}
            
            for sub_ety_pos in ety_pronunc_pos_node.parent.find_all('h4'):
                if sub_ety_pos.text.lower() in all_pos:
                    entry_data[sub_ety_pos.text.lower()] = getDefsFromPOS(sub_ety_pos)

            if any(['etymology' in entry for entry in language_entries]): #If an etymology already exists add to new entry
                language_entries.append(entry_data)
            else:
                language_entries[0].update(entry_data)
                
        # Need to see if there are sub items of this etymology
        elif node_class == 'pronunciation':
            ipa_nodes = ety_pronunc_pos_node.parent.select('span.IPA') #dataquest.io/blog/web-scraping-tutorial-python/
            if ipa_nodes: #Only add pronunciation if there are ipa_nodes
                node_data = ipa_nodes[0].text
                language_entries[0]['pronunciation'] = node_data

        elif node_class in all_pos: # Here are the definitions
            language_entries[0][node_class] = getDefsFromPOS(ety_pronunc_pos_node)

        else: # Skip all other node_classes
            continue

        page_data[language] = language_entries #Add all the language entries to the language 
print (json.dumps(page_data, indent=4))

{
    "term": "cat",
    "English": [
        {
            "pronunciation": "/k\u00e6t/",
            "etymology": "From Middle English cat, catte, from Old English catt (\u201cmale cat\u201d), catte (\u201cfemale cat\u201d), from Proto-Germanic *kattuz. ",
            "noun": [
                "A domesticated subspecies (Felis silvestris catus) of feline animal, commonly kept as a house pet. [from 8thc.]",
                "Any similar animal of the family Felidae, which includes lions, tigers, bobcats, etc.",
                "(offensive) A spiteful or angry woman. [from earlier 13thc.]",
                "An enthusiast or player of jazz.",
                "(slang) A person (usually male).",
                "(slang) A prostitute. [from at least early 15thc.]",
                "(nautical) A strong tackle used to hoist an anchor to the cathead of a ship.",
                "(chiefly nautical) Short form of cat-o'-nine-tails.",
                "(archaic) A sturdy merchant sailing vessel (n

### Parsed response into SQL

- Languages (1,2,3,4)
    - Etymology (1,2,3,4)
    - Pronunciation (1,2,3,4)
    - POS (1,2,3,4)
        - Definitions (1,2,3,4

Need the scraper to determine the number of entries:
- Hand (English) has one entry with multiple POS and one etymology
- Cat (English) has 9 entries each with one etymology and multiple POS
- Are there any entries with multiple etymologies?

In [115]:
with open('/home/ubuntu/scrapy/wiktionary_scraper/output/test3.json', 'r') as results:
    for line in results:
        this_data = json.loads(line)
        if this_data['term'] == 'cat': break
        
print('SCRAPED DATA')
print(json.dumps(this_data, indent=4))
        
word = this_data['term']
del this_data['term']

for lang, values in this_data.items():
    print()
    print('STORED DATA')
    language = lang
    
    cursor.execute(f'SELECT _id FROM etymologies e, languages l WHERE word = "{word}" and language_name = "{language}" and e.language_code = l.language_code')
    ety_id = cursor.fetchone()[0]; print('etymology:', ety_id)
    
    # Check for matching to existing entries here
    cursor.execute(f'SELECT entry_id FROM entry_connections WHERE etymology_id = {ety_id}')
    entry_ids = [item[0] for item in cursor.fetchall()]; print('entry_ids:', entry_ids)
    
    # Insert new entry ID
#     new_entry_ids
    # Add each element to SQL with that entry ID
    
    
    #What if multiple entries

    cursor.execute(f'SELECT definition FROM entry_definitions WHERE entry_id = {entry_ids[0]}')
    definitions = cursor.fetchall(); print('definitions:', definitions)

    cursor.execute(f'SELECT etymology FROM entry_etymologies WHERE entry_id= {entry_ids[0]}')
    etymologies = cursor.fetchall(); print('etymologies:', etymologies)

    cursor.execute(f'SELECT part_of_speech FROM entry_pos WHERE entry_id = {entry_ids[0]}')
    pos = cursor.fetchall(); print('pos:', pos)

    cursor.execute(f'SELECT pronunciations FROM entry_pronunciations WHERE entry_id = {entry_ids[0]}')
    pronunciations = cursor.fetchall(); print('pronunciations:', pronunciations)
    break

SCRAPED DATA
{
    "term": "cat",
    "English": [
        {
            "pronunciation": "/k\u00e6t/",
            "etymology": "From Middle English cat, catte, from Old English catt (\u201cmale cat\u201d), catte (\u201cfemale cat\u201d), from Proto-Germanic *kattuz. ",
            "noun": [
                "A domesticated subspecies (Felis silvestris catus) of feline animal, commonly kept as a house pet. [from 8thc.]",
                "Any similar animal of the family Felidae, which includes lions, tigers, bobcats, etc.",
                "(offensive) A spiteful or angry woman. [from earlier 13thc.]",
                "An enthusiast or player of jazz.",
                "(slang) A person (usually male).",
                "(slang) A prostitute. [from at least early 15thc.]",
                "(nautical) A strong tackle used to hoist an anchor to the cathead of a ship.",
                "(chiefly nautical) Short form of cat-o'-nine-tails.",
                "(archaic) A sturdy merchant sail

In [ ]:
### Parsed response into SQL

Need the scraper to determine the number of entries:
- Hand (English) has one entry with multiple POS and one etymology
- Cat (English) has 9 entries each with one etymology and multiple POS
- Are there any entries with multiple etymologies?

with open('/home/ubuntu/scrapy/wiktionary_scraper/output/test3.json', 'r') as results:
    for line in results:
        this_data = json.loads(line)
        if this_data['term'] == 'cat': break
        
print('SCRAPED DATA')
print(json.dumps(this_data, indent=4))
        
word = this_data['term']
del this_data['term']

for lang, values in this_data.items():
    print()
    print('STORED DATA')
    language = lang
    
    cursor.execute(f'SELECT _id FROM etymologies e, languages l WHERE word = "{word}" and language_name = "{language}" and e.language_code = l.language_code')
    ety_id = cursor.fetchone()[0]; print('etymology:', ety_id)
    
    # Check for matching to existing entries here
    cursor.execute(f'SELECT entry_id FROM entry_connections WHERE etymology_id = {ety_id}')
    entry_ids = [item[0] for item in cursor.fetchall()]; print('entry_ids:', entry_ids)
    
    # Insert new entry ID
#     new_entry_ids
    # Add each element to SQL with that entry ID
    
    
    #What if multiple entries

    cursor.execute(f'SELECT definition FROM entry_definitions WHERE entry_id = {entry_ids[0]}')
    definitions = cursor.fetchall(); print('definitions:', definitions)

    cursor.execute(f'SELECT etymology FROM entry_etymologies WHERE entry_id= {entry_ids[0]}')
    etymologies = cursor.fetchall(); print('etymologies:', etymologies)

    cursor.execute(f'SELECT part_of_speech FROM entry_pos WHERE entry_id = {entry_ids[0]}')
    pos = cursor.fetchall(); print('pos:', pos)

    cursor.execute(f'SELECT pronunciations FROM entry_pronunciations WHERE entry_id = {entry_ids[0]}')
    pronunciations = cursor.fetchall(); print('pronunciations:', pronunciations)
    break